In [1]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9


/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
F = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1,10,100,1000,10000,100000]
flux = np.array([F2fluxconverter(F_) for F_ in F])
plt.plot(flux,F)
plt.xscale("log")
plt.yscale("log")
plt.grid()
plt.xlabel("flux")
plt.ylabel("F")
plt.title("Flux vs F")
plt.show()


In [ ]:
LR = 1e-2
max_shift = 64
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
lmbda = 1e-9
noise = 1e9
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_)
model.solve_multigrid()
model.print_time()



----------- s = 9 -----------
----------- s = 8 -----------
----------- s = 7 -----------
----------- s = 6 -----------


KeyboardInterrupt: 

In [ ]:
if max_shift == 32:
    overlap = 75
elif max_shift == 64:
    overlap = 50
else:
    overlap = 25
from src.utils.manage_data import save_data,unwrap_2d,extract_data
path = save_data(model = model,
          image_path= "samples/potential.npy",
          metrics = ['csim','loss','frc',"image"],
          device = "cuda",
          max_scale = 9,
          overlap = overlap,
          spline_type="cpwc",
          lambda_ = lmbda,
          noise_type="poisson",
          noise = noise,
          loop = "mrgd")

In [ ]:
image = np.load(path + "_image.npy")

In [ ]:
plt.imshow(image)
plt.axis("off")

In [ ]:

frc = np.load(path + "_frc.npy")

In [ ]:
plt.figure(dpi = 200)
plt.plot(frc)
plt.grid()

In [ ]:
frc1 = np.load("data/cpwc_overlap50_mrgd_lambda1e-09_noise_typepoisson_noise10.0_frc.npy")
frc2 = np.load("data/cpwc_overlap50_mrgd_lambda0.0001_noise_typepoisson_noise10.0_frc.npy")
frc3 = np.load("data/cpwc_overlap50_mrgd_lambda0.001_noise_typepoisson_noise10.0_frc.npy")
frc4 = np.load("data/cpwc_overlap50_mrgd_lambda0.01_noise_typepoisson_noise10.0_frc.npy")
frc5 = np.load("data/cpwc_overlap50_mrgd_lambda0.1_noise_typepoisson_noise10.0_frc.npy")
fourier_k_range = np.fft.fftfreq(512) / 0.25
fourier_k_range = fourier_k_range[0:256] * 10 
plt.figure(dpi = 300,figsize=(14,10))
plt.plot(fourier_k_range,frc1)
plt.plot(fourier_k_range,frc2)
plt.plot(fourier_k_range,frc3)
plt.plot(fourier_k_range,frc4)
plt.plot(fourier_k_range,frc5)
plt.grid()
plt.legend(["Overlap 50 w/ Noise 1e1 lambda = 1e-9",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-4",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-3",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-2",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-1"])
plt.title("FRC")
plt.xlabel("Dist From Center (nm^-1)")
plt.ylabel("Fourier Ring Correlation")
plt.show()


In [ ]:
img1 = np.load("data/cpwc_overlap50_mrgd_lambda1e-09_noise_typepoisson_noise10.0_image.npy")
img2 = np.load("data/cpwc_overlap50_mrgd_lambda0.0001_noise_typepoisson_noise10.0_image.npy")
img3 = np.load("data/cpwc_overlap50_mrgd_lambda0.001_noise_typepoisson_noise10.0_image.npy")
img4 = np.load("data/cpwc_overlap50_mrgd_lambda0.01_noise_typepoisson_noise10.0_image.npy")
img5 = np.load("data/cpwc_overlap50_mrgd_lambda0.1_noise_typepoisson_noise10.0_image.npy")
plt.figure(dpi = 300,figsize=(14,10))
plt.subplot(2,3,1)
plt.imshow(img1, cmap = "gray")
plt.axis("off")
plt.title("Overlap 50 w/ Noise 1e1 lambda = 1e-9")
plt.subplot(2,3,2)
plt.imshow(img2, cmap = "gray")
plt.axis("off")
plt.title("Overlap 50 w/ Noise 1e1 lambda = 1e-4")
plt.subplot(2,3,3)
plt.imshow(img3, cmap = "gray")
plt.axis("off")
plt.title("Overlap 50 w/ Noise 1e1 lambda = 1e-3")
plt.subplot(2,3,4)
plt.imshow(img4, cmap = "gray")
plt.axis("off")
plt.title("Overlap 50 w/ Noise 1e1 lambda = 1e-2")
plt.subplot(2,3,5)
plt.imshow(img5, cmap = "gray")
plt.axis("off")
plt.title("Overlap 50 w/ Noise 1e1 lambda = 1e-1")
plt.tight_layout()
plt.show()


In [ ]:
csim1 = np.load("data/cpwc_overlap50_mrgd_lambda1e-09_noise_typepoisson_noise10.0_csim.npy")
csim2 = np.load("data/cpwc_overlap50_mrgd_lambda0.0001_noise_typepoisson_noise10.0_csim.npy")
csim3 = np.load("data/cpwc_overlap50_mrgd_lambda0.001_noise_typepoisson_noise10.0_csim.npy")
csim4 = np.load("data/cpwc_overlap50_mrgd_lambda0.01_noise_typepoisson_noise10.0_csim.npy")
csim5 = np.load("data/cpwc_overlap50_mrgd_lambda0.1_noise_typepoisson_noise10.0_csim.npy")

plt.figure(dpi = 300,figsize=(14,10))
plt.semilogy(1-csim1)
plt.semilogy(1-csim2)
plt.semilogy(1-csim3)
plt.semilogy(1-csim4)
plt.semilogy(1-csim5)
plt.grid()
plt.legend(["Overlap 50 w/ Noise 1e1 lambda = 1e-9",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-4",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-3",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-2",
            "Overlap 50 w/ Noise 1e1 lambda = 1e-1"])
plt.title("1 - Cosine Similarity")
plt.xlabel("Iterations")
plt.ylabel("Cosine Similarity")
plt.show()
